In [ ]:
from init_notebook import *
from src.models.attention import Attention1d

In [ ]:
def plot_convs(ch: int = 64, len: int = 100):
    def _conv(ch_in, ch_out):
        c = nn.Conv1d(
            ch_in, ch_out, 
            #kernel_size=3, padding=1,
            kernel_size=13, padding=6 * dilation, 
            dilation=dilation,
            bias=False,
        )
        with torch.no_grad():
            c.weight[:] = c.weight * 4
        return c
        
    states = []
    state = torch.zeros(1, ch, len)
    state[0, :, -1:] = 1 #torch.rand(1, ch, 1)
    states.append(state)
    for i in range(10):
        dilation = 1 + i % 3
        if i == 3:
            state = _conv(len, len)(state.permute(0, 2, 1)).permute(0, 2, 1)
        else:
            state = _conv(ch, ch)(state)
        states.append(state)
        if i == -1:
            c = nn.Conv1d(ch, ch * 2, 1)
            #state = state
            y = c(state).permute(0, 2, 1)
            q, k = torch.split(y, y.shape[-1] // 2, dim=-1)
            a = Attention1d()
            state = 10*a(q, k, state.permute(0, 2, 1)).permute(0, 2, 1)
            states.append(state)

    max_len = max(s.shape[-1] for s in states)
    grid = [
        torch.cat([torch.ones((s.shape[-2], max_len - s.shape[-1])) * .5, s[0]], dim=-1)[:3]
        .abs().unsqueeze(-2)
        for s in states                                 
    ]
    display(VF.to_pil_image(resize(make_grid(grid, nrow=1, padding=2), 4)))
    
plot_convs()

In [ ]:
F.cross_entropy(
    torch.zeros(16, 100),
    torch.zeros(16, dtype=torch.long),
)